# 1. Import Dependencies

In [48]:
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
from imblearn.over_sampling import SMOTE
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import Callback

# 2. Data Preparation

In [49]:
df = pd.read_csv('push_up_hand_position_fix.csv')

In [50]:
jumlah_kelas_0 = df[df['class'] == 0].shape[0]
jumlah_kelas_1 = df[df['class'] == 1].shape[0]

print("Jumlah data kelas 0:", jumlah_kelas_0)
print("Jumlah data kelas 1:", jumlah_kelas_1)

Jumlah data kelas 0: 2674
Jumlah data kelas 1: 3319


In [51]:
X = df.drop('class', axis=1)
y = df['class']

In [52]:
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

In [53]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [54]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# 3. Modelling

In [66]:
model = tf.keras.Sequential([
    layers.Dense(64, input_dim=132, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1, activation='sigmoid')  
])


In [67]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [68]:
class AccuracyStopCallback(Callback):
    def __init__(self, target_accuracy=0.99):
        super(AccuracyStopCallback, self).__init__()
        self.target_accuracy = target_accuracy
        self.val_accuracy = target_accuracy

    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') >= self.target_accuracy and logs.get('val_accuracy') >= self.target_accuracy:
            print(f"\nReached target accuracy of {self.target_accuracy}, stopping training!")
            self.model.stop_training = True

In [69]:
callback = AccuracyStopCallback(target_accuracy=0.97)

In [70]:
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2, callbacks=[callback])

Epoch 1/20
161/166 [============================>.] - ETA: 0s - loss: 0.9503 - accuracy: 0.9740
Reached target accuracy of 0.97, stopping training!
166/166 [==============================] - 2s 11ms/step - loss: 0.9362 - accuracy: 0.9748 - val_loss: 0.5429 - val_accuracy: 0.9940


In [71]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model_head_position.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpbjgp51o2\assets


INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpbjgp51o2\assets


# 4. Testing

In [72]:
mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose

landmarks = ["class"]
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

vid_path = "test.mp4"
cap = cv2.VideoCapture(vid_path)
current_stage = ''

# Initiate Pose Model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    # Streaming the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Resize the frame to the desired window size
        frame = cv2.resize(frame, (720, 600))

        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detections
        results = pose.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))
        try:
            if results.pose_landmarks:
                row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
                row = np.expand_dims(row, axis=0)  # Add a batch dimension
                prediction = model.predict(row)
                print(f'Result: {prediction[0][0]}')
                
                if prediction[0][0] > 0.5:
                    current_stage = 'Correct'
                elif prediction[0][0] <= 0.5:
                    current_stage = 'Wrong'

                cv2.rectangle(image, (0, 0), (250, 60), (245, 117, 16), -1)

                cv2.putText(image, 'CLASS', (95, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, current_stage, (95, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                cv2.putText(image, 'PROB', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(prediction[0][0], 2)), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        except Exception as e:
            print(f"Error: {e}")


        # Stream video result
        cv2.imshow("Raw Cam Feed", image)

        # Press 'q' to stop the video
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


Result: 0.6369575262069702
Result: 0.6364229321479797
Result: 0.6393422484397888
Result: 0.6375902891159058
Result: 0.6323860287666321
Result: 0.6324885487556458
Result: 0.6340835094451904
Result: 0.6394277215003967
Result: 0.6523882746696472
Result: 0.6544374227523804
Result: 0.6510941982269287
Result: 0.6403170228004456
Result: 0.6389591097831726
Result: 0.6409387588500977
Result: 0.6395072937011719
Result: 0.6334919929504395
Result: 0.6263447403907776
Result: 0.6190959811210632
Result: 0.5975582599639893
Result: 0.5745760798454285
Result: 0.5730619430541992
Result: 0.5602943301200867
Result: 0.5499590039253235
Result: 0.5400972962379456
Result: 0.5286163687705994
Result: 0.5275354981422424
Result: 0.5086233615875244
Result: 0.5106104612350464
Result: 0.5007687211036682
Result: 0.49781128764152527
Result: 0.5011120438575745
Result: 0.5001294016838074
Result: 0.501552402973175
Result: 0.4965318739414215
Result: 0.49892622232437134
Result: 0.49439501762390137
Result: 0.4905677735805511